In [ ]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..','..', 'dataset'))
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from torchvision.models import vit_b_16
from TwitterImageDataset import TwitterImageDataset


transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])


dataset = TwitterImageDataset("../../AutoDep_Master", transform=transform)


train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model = vit_b_16(pretrained=True)

num_features = model.heads.head.in_features
model.heads.head = nn.Linear(num_features, 2)  # 2 classes


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

print('Training has started...')

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")


/Users/ardakabadayi/Desktop/Code/Syracuse/CIS662(IntroToMl)/env/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/ardakabadayi/Desktop/Code/Syracuse/CIS662(IntroToMl)/env/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /Users/ardakabadayi/Desktop/Code/Syracuse/CIS662(IntroToMl)/env/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/ardakabadayi/Desktop/Code/Syracuse/CIS662(IntroToMl)/env/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/ardakabadayi/Desktop/Code/Syracuse/CIS662(IntroToMl)/env/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/ardakabadayi/Desktop/Code/Syracuse/CIS662(IntroToMl)/env/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/ardakabadayi/Desktop/Code/Sy

../../AutoDep_Master/control_group/users
../../AutoDep_Master/diagnosed_group/users


/Users/ardakabadayi/Desktop/Code/Syracuse/CIS662(IntroToMl)/env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ardakabadayi/Desktop/Code/Syracuse/CIS662(IntroToMl)/env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /Users/ardakabadayi/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████████████████████████████| 330M/330M [00:04<00:00, 73.2MB/s]


Training has started...


KeyboardInterrupt: 